In [1]:

!pip install -q findspark

import findspark
findspark.init('/home/ec2-user/spark-2.4.5-bin-hadoop2.7')



In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizerModel, IDFModel, StandardScalerModel, Tokenizer
from pyspark.ml.classification import LogisticRegressionModel


from urllib.parse import unquote

In [3]:
APP_NAME = "BigData"
conf = pyspark.SparkConf().setAll([ ('spark.app.name', APP_NAME),
                                    ('spark.executor.memory', '8g'),
                                    ('spark.cores.max', '2'),
                                    ('spark.driver.memory','8g'),
                                   ('spark.master', 'local[2]')])
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)
sc

<SparkContext master=local[2] appName=BigData>

In [8]:
pyspark.SparkConf().getAll()

[('spark.driver.memory', '8g'),
 ('spark.app.name', 'BigData'),
 ('spark.cores.max', '2'),
 ('spark.executor.memory', '8g'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'local[2]')]

BlockRDD[127] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[128] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[129] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[130] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[131] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[132] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[133] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[134] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[135] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[136] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[137] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[138] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[139] at socketTextStream at NativeMethodAccessorImpl.java:0
No data

BlockRDD[234] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[235] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[236] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[237] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[238] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[239] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[240] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[241] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[242] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[243] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[244] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[245] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[246] at socketTextStream at NativeMethodAccessorImpl.java:0
No data

BlockRDD[341] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[342] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[343] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[344] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[345] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[346] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[347] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[348] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[349] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[350] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[351] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[352] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[353] at socketTextStream at NativeMethodAccessorImpl.java:0
No data

BlockRDD[448] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[449] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[450] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[451] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[452] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[453] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[454] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[455] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[456] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[457] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[458] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[459] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[460] at socketTextStream at NativeMethodAccessorImpl.java:0
No data

BlockRDD[555] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[556] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[557] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[558] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[559] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[560] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[561] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[562] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[563] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[564] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[565] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[566] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[567] at socketTextStream at NativeMethodAccessorImpl.java:0
No data

In [4]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = ''
    for i in range(0,len(payload)-n + 1):
        ngrams += payload[i:i+n]+ ' '
    return ngrams[:-1]

ngrams = udf(to_ngram, StringType())
tokenizer = Tokenizer.load('models/Tokenizer')
vectorizer = CountVectorizerModel.load('models/Vectorizer')
idf_model = IDFModel.load('models/idf')
scalerModel = StandardScalerModel.load('models/scalerModel')
model = LogisticRegressionModel.load('models/Logistic_Regression_Model')

In [5]:
queries = sc.parallelize(['Alice','bigdata',"hellworld","select* from students where '1'='1","<sctipt>alert('hacked');</sctipt>"]).map(lambda q: Row(payload=q))
sample_df = sqlc.createDataFrame(queries)

sample_df = sample_df.withColumn('ngrams', ngrams(sample_df['payload']))
sample_df = tokenizer.transform(sample_df)
sample_df = vectorizer.transform(sample_df)
sample_df = idf_model.transform(sample_df)
sample_df = scalerModel.transform(sample_df).cache()
preds = model.transform(sample_df)
sample_df.select('payload','scaledFeatures').show()

+--------------------+--------------------+
|             payload|      scaledFeatures|
+--------------------+--------------------+
|               Alice|(4472,[34,42,53,1...|
|             bigdata|(4472,[24,43,169,...|
|           hellworld|(4472,[23,75,86,9...|
|select* from stud...|(4472,[0,13,17,19...|
|<sctipt>alert('ha...|(4472,[3,6,7,13,2...|
+--------------------+--------------------+



In [6]:
preds = model.transform(sample_df)
preds.select('payload','prediction').show(truncate=False)

+----------------------------------+----------+
|payload                           |prediction|
+----------------------------------+----------+
|Alice                             |0.0       |
|bigdata                           |0.0       |
|hellworld                         |0.0       |
|select* from students where '1'='1|1.0       |
|<sctipt>alert('hacked');</sctipt> |1.0       |
+----------------------------------+----------+



In [7]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = ''
    for i in range(0,len(payload)-n + 1):
        ngrams += payload[i:i+n]+ ' '
    return ngrams[:-1]

ngrams = udf(to_ngram, StringType())

# define a function to compute sentiments of the received tweets
def get_prediction(queries):
    print(queries)
    try:
        queries = queries.map(lambda w: Row(payload=w))
        queries = sqlc.createDataFrame(queries)
        queries = queries.withColumn('ngrams', ngrams(queries['payload']))
        queries = tokenizer.transform(queries)
        queries = vectorizer.transform(queries)
        queries = idf_model.transform(queries)
        queries = scalerModel.transform(queries)
        preds = model.transform(queries)
        preds.select('payload','prediction').show()
    except : 
        print('No data')
    

ssc = StreamingContext(sc, batchDuration= 3)
lines = ssc.socketTextStream("localhost", 9999)

lines.foreachRDD(get_prediction)

ssc.start()             

ssc.awaitTermination()

BlockRDD[58] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[59] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[60] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[61] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[62] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[63] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[64] at socketTextStream at NativeMethodAccessorImpl.java:0
+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|asdfdfasdfasdfads...|       0.0|
+--------------------+----------+

BlockRDD[76] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[77] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[78] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
BlockRDD[79] at socketTextStream at NativeMethodAccessorImpl.java:0
No data
B

KeyboardInterrupt: 